In [5]:
import os
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import math
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import f1_score, roc_auc_score

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, LSTM
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.optimizers import Adam
from keras.models import Sequential
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.layers import concatenate
from keras.callbacks import *
from keras.utils import to_categorical


In [6]:
import keras
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.Session(config=config)
set_session(sess)

keras.backend.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [9]:
#google word2vec
from gensim.models import KeyedVectors as wv
word_vectors = wv.load_word2vec_format('./input/embeddings/GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin', binary = True)
EMBEDDING_DIM =300

##glove
# embeddings_index = {}
# f = open(os.path.join(GLOVE_DIR, './input/embeddings/glove.840B.300d/glove.840B.300d.txt'))
# for line in f:
#     values = line.split()
#     word = values[0]
#     coefs = np.asarray(values[1:], dtype='float32')
#     embeddings_index[word] = coefs
# f.close()
# EMBEDDING_DIM =300

# print('Found %s word vectors.' % len(embeddings_index))

/home/samarth/anaconda3/envs/tensorflow36/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


TypeError: 'int' object is not iterable

In [10]:
df_train = pd.read_csv("./input/train.csv")
df_test = pd.read_csv("./input/test.csv")
print("Train shape : ",df_train.shape)
print("Test shape : ",df_test.shape)

df_train_pos = df_train[df_train['target']==1]
df_train_neg = df_train[df_train['target']==0].sample(len(df_train_pos)*2,random_state=1)
df_train = pd.concat([df_train_pos,df_train_neg])
df_train = df_train.sample(frac=1,random_state=1)
print("Train shape : ",df_train.shape)

df_train.dropna(inplace=True)
df_test.dropna(inplace=True)

MAX_SEQUENCE_LENGTH=150


tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_train['question_text'])
sequences = tokenizer.texts_to_sequences(df_train['question_text'])

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

train_X = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, padding="post")
train_Y = df_train['target']
# labels = to_categorical(np.asarray(labels))
# print('Shape of data tensor:', data.shape)
# print('Shape of label tensor:', labels.shape)

# split the data into a training set and a validation set
# indices = np.arange(data.shape[0])
# np.random.shuffle(indices)
# data = data[indices]
# labels = labels[indices]
# nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

# x_train = data[:-nb_validation_samples]
# y_train = labels[:-nb_validation_samples]
# x_val = data[-nb_validation_samples:]
# y_val = labels[-nb_validation_samples:]

Train shape :  (1306122, 3)
Test shape :  (56370, 2)
Train shape :  (242430, 3)
Found 87822 unique tokens.


In [17]:
train_X[4]

# a = word_vectors['as12sad34']
word_index

{'the': 1,
 'to': 2,
 'is': 3,
 'a': 4,
 'in': 5,
 'what': 6,
 'of': 7,
 'and': 8,
 'do': 9,
 'why': 10,
 'are': 11,
 'i': 12,
 'how': 13,
 'for': 14,
 'you': 15,
 'it': 16,
 'can': 17,
 'that': 18,
 'with': 19,
 'have': 20,
 'if': 21,
 'or': 22,
 'my': 23,
 'on': 24,
 'be': 25,
 'people': 26,
 'does': 27,
 'they': 28,
 'as': 29,
 'when': 30,
 'from': 31,
 'so': 32,
 'not': 33,
 'an': 34,
 'their': 35,
 'should': 36,
 'would': 37,
 'like': 38,
 'who': 39,
 'get': 40,
 'your': 41,
 'will': 42,
 'about': 43,
 'there': 44,
 'by': 45,
 'which': 46,
 'some': 47,
 'did': 48,
 'at': 49,
 'best': 50,
 'was': 51,
 'we': 52,
 'all': 53,
 'any': 54,
 'this': 55,
 'india': 56,
 'me': 57,
 'more': 58,
 'has': 59,
 'trump': 60,
 'think': 61,
 'women': 62,
 'good': 63,
 'than': 64,
 'but': 65,
 'many': 66,
 'most': 67,
 'one': 68,
 'them': 69,
 'he': 70,
 'other': 71,
 'make': 72,
 'after': 73,
 "don't": 74,
 'quora': 75,
 'much': 76,
 'being': 77,
 'men': 78,
 'just': 79,
 'his': 80,
 'us': 81,
 'wh

In [ ]:

df_train.iloc[4]['question_text']

In [ ]:
lengths = []
for sent in df_train['question_text']:
#     print(sent)
    lengths.append(len(sent))
#     if max(sent)>50:
#         print(sent)
print(np.mean(lengths))
print(np.median(lengths))
print(np.std(lengths))
print(np.max(lengths))
plt.hist(lengths,density=False)

In [20]:
#for google word2vec

embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    try:
        word_vector = word_vectors[word]
#     if word_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = word_vector
    except KeyError:
        continue

In [21]:
# print(train_X_raw[0])
print(train_X.shape)
print(train_Y.shape)
# print(tokenizer.word_index)

(242430, 150)
(242430,)


In [23]:
##model definition
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False))


# model.add(CuDNNLSTM(75, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Bidirectional(CuDNNLSTM(75, return_sequences=True,), input_shape=(train_X.shape[1], EMBEDDING_DIM)))
model.add(Flatten())
model.add(Dense(100,activation='relu'))
model.add(Dense(1, activation='softmax'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 150, 300)          26346900  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 150, 150)          226200    
_________________________________________________________________
flatten_1 (Flatten)          (None, 22500)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               2250100   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 28,823,301
Trainable params: 2,476,401
Non-trainable params: 26,346,900
_________________________________________________________________
None


In [26]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])  

In [27]:
model.fit(train_X, train_Y, epochs=100,batch_size=500,validation_split=0.2)

Train on 193944 samples, validate on 48486 samples
Epoch 1/100
193944/193944 [==============================] - 26s 134us/step - loss: 10.6158 - acc: 0.3341 - val_loss: 10.6779 - val_acc: 0.3302
Epoch 2/100
193944/193944 [==============================] - 26s 131us/step - loss: 10.6158 - acc: 0.3341 - val_loss: 10.6779 - val_acc: 0.3302
Epoch 3/100
193944/193944 [==============================] - 26s 132us/step - loss: 10.6158 - acc: 0.3341 - val_loss: 10.6779 - val_acc: 0.3302
Epoch 4/100
193944/193944 [==============================] - 26s 133us/step - loss: 10.6158 - acc: 0.3341 - val_loss: 10.6779 - val_acc: 0.3302
Epoch 5/100
193944/193944 [==============================] - 26s 133us/step - loss: 10.6158 - acc: 0.3341 - val_loss: 10.6779 - val_acc: 0.3302
Epoch 6/100
193944/193944 [==============================] - 26s 136us/step - loss: 10.6158 - acc: 0.3341 - val_loss: 10.6779 - val_acc: 0.3302
Epoch 7/100
193944/193944 [==============================] - 26s 134us/step - loss: 1

KeyboardInterrupt: 

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Conv2D, Flatten

model=Sequential()
model.add(Conv2D(10,(3,3)))
model.add(Conv2D(10,(3,3)))
model.add(Flatten())
model.add(Dense(20))
model.add(Dense(10))

model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])
# model.build()
# model.summary()

In [ ]:
import tensorflow as tf; print(tf.__version__)
import keras; print(keras.__version__)

In [ ]:
def load_glove(word_index):
    EMBEDDING_FILE = '../input/embeddings/glove.840B.300d/glove.840B.300d.txt'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]

    # word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector
            
    return embedding_matrix 

In [ ]:
import string
letter_count = dict(zip(string.ascii_lowercase, [0]*26))

In [ ]:
{chr(i+96):i for i in range(1,27)}

In [ ]:
## Tokenize the sentences
# tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True, char_level=True)
# tokenizer.fit_on_texts(list(train_X_raw))

# train_X = tokenizer.texts_to_sequences(train_X_raw)
# test_X = tokenizer.texts_to_sequences(test_X_raw)

# for sent in train_X_raw:
#     s=[]
#     for char in sent.lower():
#         if char in char_dict.keys():
#             s.append(char_dict[char])
#     if s!=[]:
#         train_X_filt.append(s)
#     else:
#         print(sent)

# ## Get the target values
# # train_Y = df_train['target'].values

# test_X_filt = []
# for sent in test_X_raw:
#     s=[]
#     for char in sent.lower():
#         if char in char_dict.keys():
#             s.append(char_dict[char])
#     test_X_filt.append(s)